# Input Processing

In [1]:
import re
pattern = re.compile(r"Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.*)")

In [2]:
data = open("data/16.txt", "r").read()

In [3]:
matches = pattern.findall(data)

In [4]:
from dataclasses import dataclass
@dataclass
class Valve:
    name: str
    flow_rate: int
    edges: list[str]

In [5]:
valve_map: dict[str, Valve] = {}

In [6]:
for match in matches:
    valve_map[match[0]] = Valve(match[0], int(match[1]), match[2].split(", "))

# Part 1
## Precompute paths (BFS)

In [7]:
def generate_path(from_valve: str, to_valve: str):
    stack: list[tuple[str]] = [(from_valve,)]
    visited = set()
    while len(stack) > 0:
        hand = stack.pop(0)

        if hand[-1] == to_valve:
            return hand

        for edge in valve_map[hand[-1]].edges:
            if edge in visited:
                continue
            stack.append(hand + (edge,))
            visited.add(edge)

In [8]:
no_flow = []
for valve in valve_map.keys():
    if valve_map[valve].flow_rate == 0:
        no_flow.append(valve)

In [9]:
path_map: dict[str, dict[str, tuple[str]]] = {}
for a in valve_map.keys():
    if a in no_flow and a != "AA":
        continue
    path_map[a] = {}
    for b in valve_map.keys():
        if b in no_flow:
            continue
        path_map[a][b] = generate_path(a,b)[1:]

## DFS (by path, full search)

In [10]:
def find_max_released(max_time=30):
    max_released = 0
    # Stack of (current_position, opened_valves, released, time_passed)
    stack: list[tuple[str, tuple[str], int, int]] = [("AA", tuple(no_flow), 0, 1)]
    while len(stack):
        # Pop
        current_position, opened_valves, released, time_passed = stack.pop()
        # Process
        max_released = max(released, max_released)
        # Push
        if len(opened_valves) == len(valve_map) or time_passed >= max_time:
            continue
        for target in path_map[current_position].keys():
            if target in opened_valves:
                continue
            end_time = min(time_passed + len(path_map[current_position][target]), max_time)
            next_released = released + ((max_time - end_time) * valve_map[target].flow_rate if end_time <= max_time else 0)
            stack.append((
                target,
                opened_valves + (target,),
                next_released,
                end_time + 1
            ))
    return max_released

In [11]:
print(find_max_released(30))

1789


# Part 2
## DFS (by path, full search) (very slow)

In [12]:
def find_max_released_elephant(max_time=30):
    max_released = 0
    # Stack of (current_position, elephant_position, opened_valves, released, time_passed, elephant_passed)
    stack: list[tuple[str, str, tuple[str], int, int, int]] = [("AA", "AA", tuple(no_flow), 0, 1, 1)]
    while len(stack):
        # Pop
        current_position, elephant_position, opened_valves, released, time_passed, elephant_passed = stack.pop()
        # Process
        max_released = max(released, max_released)
        # Push
        lowest_time = min(time_passed, elephant_passed)
        choice = "you" if lowest_time == time_passed else "elephant"
        if len(opened_valves) == len(valve_map) or lowest_time >= max_time:
            continue
        for target in path_map[current_position if choice == "you" else elephant_position].keys():
            if target in opened_valves:
                continue
            end_time = min((time_passed if choice == "you" else elephant_passed) +
                           len(path_map[current_position if choice == "you" else elephant_position][target]), max_time)
            next_released = released + ((max_time - end_time) * valve_map[target].flow_rate if end_time <= max_time else 0)
            stack.append((
                target if choice == "you" else current_position,
                target if choice == "elephant" else elephant_position,
                opened_valves + (target,),
                next_released,
                end_time + 1 if choice == "you" else time_passed,
                end_time + 1 if choice == "elephant" else elephant_passed,
            ))
    return max_released

In [13]:
print(find_max_released_elephant(26))

2496
